In [ ]:
import sys
import json
import os
sys.path.append('..')
from google.cloud import bigquery
import geopandas as gpd

from geodoc_loader.download.core import download_data_from_sources
from geodoc_loader.download.validate import collect_columns_for_sources, find_files_with_extension

In [31]:
config_path = '../bdzp_config.json'
target_path = 'bdzp_temp'

with open(config_path, 'r') as f:
    config = json.load(f)

client = bigquery.Client()
project_id = client.project

### download data from sources

In [ ]:
results = download_data_from_sources(config['sources'], target_path)
results

Successfully extracted bdzp_temp/parki_narodowe.zip to bdzp_temp/parki_narodowe
Deleted ZIP file: bdzp_temp/parki_narodowe.zip
Successfully downloaded and unzipped parki_narodowe


Successfully extracted bdzp_temp/parki_krajobrazowe.zip to bdzp_temp/parki_krajobrazowe
Deleted ZIP file: bdzp_temp/parki_krajobrazowe.zip
Successfully downloaded and unzipped parki_krajobrazowe


Successfully extracted bdzp_temp/rezerwaty.zip to bdzp_temp/rezerwaty
Deleted ZIP file: bdzp_temp/rezerwaty.zip
Successfully downloaded and unzipped rezerwaty


Successfully extracted bdzp_temp/obszary_chronionego_krajobrazu.zip to bdzp_temp/obszary_chronionego_krajobrazu
Deleted ZIP file: bdzp_temp/obszary_chronionego_krajobrazu.zip
Successfully downloaded and unzipped obszary_chronionego_krajobrazu


Successfully extracted bdzp_temp/specjalne_obszary_ochrony.zip to bdzp_temp/specjalne_obszary_ochrony
Deleted ZIP file: bdzp_temp/specjalne_obszary_ochrony.zip
Successfully downloaded and unzipped specjalne_obszary_ochrony


...

Successfully extracted bdzp_temp/stanowiska_dokumentacyjne.zip to bdzp_temp/stanowiska_dokumentacyjne
Deleted ZIP file: bdzp_temp/stanowiska_dokumentacyjne.zip
Successfully downloaded and unzipped stanowiska_dokumentacyjne


Successfully extracted bdzp_temp/uzytki_ekologiczne.zip to bdzp_temp/uzytki_ekologiczne
Deleted ZIP file: bdzp_temp/uzytki_ekologiczne.zip
Successfully downloaded and unzipped uzytki_ekologiczne


Successfully extracted bdzp_temp/obszary_ramsar.zip to bdzp_temp/obszary_ramsar
Deleted ZIP file: bdzp_temp/obszary_ramsar.zip
Successfully downloaded and unzipped obszary_ramsar


Successfully extracted bdzp_temp/korytarze_ekologiczne.zip to bdzp_temp/korytarze_ekologiczne
Deleted ZIP file: bdzp_temp/korytarze_ekologiczne.zip
Successfully downloaded and unzipped korytarze_ekologiczne


[{'name': 'parki_narodowe', 'status': 'success', 'error': None},
 {'name': 'parki_krajobrazowe', 'status': 'success', 'error': None},
 {'name': 'rezerwaty', 'status': 'success', 'error': None},
 {'name': 'obszary_chronionego_krajobrazu',
  'status': 'success',
  'error': None},
 {'name': 'specjalne_obszary_ochrony', 'status': 'success', 'error': None},
 {'name': 'obszary_specjalnej_ochrony', 'status': 'success', 'error': None},
 {'name': 'zespoly_przyrodniczo_krajobrazowe',
  'status': 'success',
  'error': None},
 {'name': 'stanowiska_dokumentacyjne', 'status': 'success', 'error': None},
 {'name': 'uzytki_ekologiczne', 'status': 'success', 'error': None},
 {'name': 'obszary_ramsar', 'status': 'success', 'error': None},
 {'name': 'korytarze_ekologiczne', 'status': 'success', 'error': None}]

### check columns set

In [23]:
files_data, columns_set = collect_columns_for_sources(config['sources'], target_path)

In [24]:
columns_set

['kodinspire', 'rzedna', 'nazwa', 'kod', 'geometry', 'gid']

In [25]:
files_data

[{'name': 'parki_narodowe',
  'file': 'bdzp_temp/parki_narodowe/ParkiNarodowePolygon.shp',
  'columns': ['gid', 'nazwa', 'kodinspire', 'geometry']},
 {'name': 'parki_krajobrazowe',
  'file': 'bdzp_temp/parki_krajobrazowe/ParkiKrajobrazowePolygon.shp',
  'columns': ['gid', 'nazwa', 'kodinspire', 'geometry']},
 {'name': 'rezerwaty',
  'file': 'bdzp_temp/rezerwaty/RezerwatyPolygon.shp',
  'columns': ['gid', 'nazwa', 'kodinspire', 'geometry']},
 {'name': 'obszary_chronionego_krajobrazu',
  'file': 'bdzp_temp/obszary_chronionego_krajobrazu/ObszaryChronionegoKrajobrazuPolygon.shp',
  'columns': ['gid', 'nazwa', 'kodinspire', 'geometry']},
 {'name': 'specjalne_obszary_ochrony',
  'file': 'bdzp_temp/specjalne_obszary_ochrony/SpecjalneObszaryOchronyPolygon.shp',
  'columns': ['gid', 'nazwa', 'kod', 'kodinspire', 'geometry']},
 {'name': 'obszary_specjalnej_ochrony',
  'file': 'bdzp_temp/obszary_specjalnej_ochrony/ObszarySpecjalnejOchronyPolygon.shp',
  'columns': ['gid', 'nazwa', 'kod', 'kodinsp

### setup BigQuery table

In [ ]:
from geodoc_loader.download.gcp import create_gcs_bucket, create_bigquery_dataset, create_bigquery_table

create_gcs_bucket(project_id, f"{project_id}-{config['bucket_name']}")

create_bigquery_dataset(project_id, config['dataset_name'])

create_bigquery_table(
    table_name=config['table_name'],
    collection_name=config['dataset_name'],
    project_id=project_id,
    columns_spec= config['schema'],
    additional_columns=[]
)

Bucket 'geodoc-386107-single-loads' already exists.
Dataset 'geo' already exists in project 'geodoc-386107'.
Created table geodoc-386107.geo.bdzp


'geodoc-386107.geo.bdzp'

### process & upload data

In [ ]:
from geodoc_loader.handlers.collection import load_geo_collection

load_geo_collection(target_path, config, config['table_name'], config['dataset_name'], config['bucket_name'])

Processing shapefiles in bdzp_temp:   0%|          | 0/11 [00:00<?, ?it/s]

Skipped 15 invalid geometries in bdzp_temp/uzytki_ekologiczne/UzytkiEkologicznePolygon.shp.
Skipped 4 invalid geometries in bdzp_temp/zespoly_przyrodniczo_krajobrazowe/ZespolyPrzyrodniczoKrajobrazowePolygon.shp.
Skipped 4 invalid geometries in bdzp_temp/specjalne_obszary_ochrony/SpecjalneObszaryOchronyPolygon.shp.
Skipped 1 invalid geometries in bdzp_temp/rezerwaty/RezerwatyPolygon.shp.
Skipped 1 invalid geometries in bdzp_temp/obszary_specjalnej_ochrony/ObszarySpecjalnejOchronyPolygon.shp.
GeoJSON saved to bdzp_temp/bdzp.geojson
GeoJSON saved successfully to bdzp_temp/bdzp.geojson
GeoJSON uploaded to GCS: gs://geodoc-386107-single-loads/bdzp_temp/bdzp.geojson
Loaded 12070 rows into geodoc-386107.geo.bdzp
GeoJSON loaded into BigQuery table geo.bdzp


True

In [45]:
from geodoc_loader.download.core import delete_local_temp_files

delete_local_temp_files(target_path)

Deleted local temporary directory: bdzp_temp


In [ ]:
from geodoc_loader.download.validate import load_first_n_rows_from_geojson
gdf = load_first_n_rows_from_geojson('bdzp_temp/bdzp.geojson', 100)

In [38]:
gdf

,source,filename,gid,kod,nazwa,rzedna,kodinspire,geometry
0,uzytki_ekologiczne,UzytkiEkologicznePolygon,120961,None,None,None,PL.ZIPOP.1393.UE.0414042.2224,"MULTIPOLYGON (((18.48427 53.60026, 18.48426 53..."
1,uzytki_ekologiczne,UzytkiEkologicznePolygon,107655,None,None,None,PL.ZIPOP.1393.UE.0414042.201,"POLYGON ((18.48338 53.59104, 18.48345 53.59088..."
2,uzytki_ekologiczne,UzytkiEkologicznePolygon,120962,None,None,None,PL.ZIPOP.1393.UE.0414042.2225,"POLYGON ((18.42589 53.54665, 18.42621 53.54684..."
3,uzytki_ekologiczne,UzytkiEkologicznePolygon,107657,None,None,None,PL.ZIPOP.1393.UE.0414042.213,"MULTIPOLYGON (((18.42427 53.54517, 18.42478 53..."
4,uzytki_ekologiczne,UzytkiEkologicznePolygon,120963,None,Turzycowe bobrowisko,None,PL.ZIPOP.1393.UE.0811082.419,"MULTIPOLYGON (((14.89585 51.58410, 14.89607 51..."
...,...,...,...,...,...,...,...,...
95,uzytki_ekologiczne,UzytkiEkologicznePolygon,114400,None,Wzdłuż Ilanki,None,PL.ZIPOP.1393.UE.0805043.275,"MULTIPOLYGON (((14.87708 52.35998, 14.87752 52..."
96,uzytki_ekologiczne,UzytkiEkologicznePolygon,120175,None,Obeżysko,None,PL.ZIPOP.1393.UE.1806062.488,"POLYGON ((21.91309 50.32886, 21.91298 50.32910..."
97,uzytki_ekologiczne,UzytkiEkologicznePolygon,111207,None,Droudy,None,PL.ZIPOP.1393.UE.1806062.387,"MULTIPOLYGON (((21.96403 50.37613, 21.96344 50..."
98,uzytki_ekologiczne,UzytkiEkologicznePolygon,111211,None,Bawolina,None,PL.ZIPOP.1393.UE.1806062.65,"MULTIPOLYGON (((21.88639 50.33065, 21.88661 50..."
